In [1]:
pip install pymongo

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pymongo
import pandas as pd

# Connecting the Mongodb atlas 
client = pymongo.MongoClient("mongodb://localhost:27017/")

# assigning variable for the database

db = client.Mongodb_Task 

# assigning variable for the Collection

collection = db.Student_Database


In [12]:
client.list_database_names() #Getting the list of database

['Mongodb_Task', 'Tele_records', 'admin', 'config', 'local']

In [13]:
db.list_collection_names() #Getting the list of collections

['Student_Database']

# importing student dataset

In [15]:
import pandas as pd 

df = pd.read_json('students.json', lines = True)
df

data = df.to_dict(orient = 'records')
data

[{'_id': 0,
  'name': 'aimee Zank',
  'scores': [{'score': 1.463179736705023, 'type': 'exam'},
   {'score': 11.78273309957772, 'type': 'quiz'},
   {'score': 35.8740349954354, 'type': 'homework'}]},
 {'_id': 1,
  'name': 'Aurelia Menendez',
  'scores': [{'score': 60.06045071030959, 'type': 'exam'},
   {'score': 52.79790691903873, 'type': 'quiz'},
   {'score': 71.76133439165544, 'type': 'homework'}]},
 {'_id': 2,
  'name': 'Corliss Zuk',
  'scores': [{'score': 67.03077096065002, 'type': 'exam'},
   {'score': 6.301851677835235, 'type': 'quiz'},
   {'score': 66.28344683278382, 'type': 'homework'}]},
 {'_id': 3,
  'name': 'Bao Ziglar',
  'scores': [{'score': 71.64343899778332, 'type': 'exam'},
   {'score': 24.80221293650313, 'type': 'quiz'},
   {'score': 42.26147058804812, 'type': 'homework'}]},
 {'_id': 4,
  'name': 'Zachary Langlais',
  'scores': [{'score': 78.68385091304332, 'type': 'exam'},
   {'score': 90.2963101368042, 'type': 'quiz'},
   {'score': 34.41620148042529, 'type': 'homework

In [16]:
for i in collection.find():  # Retrieving the data 
    print(i)

{'_id': 0, 'name': 'aimee Zank', 'scores': [{'score': 1.463179736705023, 'type': 'exam'}, {'score': 11.78273309957772, 'type': 'quiz'}, {'score': 35.8740349954354, 'type': 'homework'}]}
{'_id': 1, 'name': 'Aurelia Menendez', 'scores': [{'score': 60.06045071030959, 'type': 'exam'}, {'score': 52.79790691903873, 'type': 'quiz'}, {'score': 71.76133439165544, 'type': 'homework'}]}
{'_id': 2, 'name': 'Corliss Zuk', 'scores': [{'score': 67.03077096065002, 'type': 'exam'}, {'score': 6.301851677835235, 'type': 'quiz'}, {'score': 66.28344683278382, 'type': 'homework'}]}
{'_id': 3, 'name': 'Bao Ziglar', 'scores': [{'score': 71.64343899778332, 'type': 'exam'}, {'score': 24.80221293650313, 'type': 'quiz'}, {'score': 42.26147058804812, 'type': 'homework'}]}
{'_id': 4, 'name': 'Zachary Langlais', 'scores': [{'score': 78.68385091304332, 'type': 'exam'}, {'score': 90.2963101368042, 'type': 'quiz'}, {'score': 34.41620148042529, 'type': 'homework'}]}
{'_id': 5, 'name': 'Wilburn Spiess', 'scores': [{'scor

# 1. Find the student name who scored maximum scores in all (exam, quiz and homework)?

In [17]:
from pprint import pprint
stage1 = {'$unwind':'$scores'}

stage2 = {'$group':{'_id': '$scores.type','Maximum_Score':{'$max':'$scores.score'}}}


loop = collection.aggregate([stage1,stage2])

for i in loop:
    x = collection.aggregate([
        {'$unwind':'$scores'}, # decontructing a array field -scores
        {'$match' : {'$and' : [{'scores.type': i['_id']},{'scores.score' : i['Maximum_Score']}]}},
        {'$project': {'_id':0}}])
    print(list(x))

[{'name': 'Whitley Fears', 'scores': {'score': 99.77237745070993, 'type': 'homework'}}]
[{'name': 'Margart Vitello', 'scores': {'score': 99.33685767140612, 'type': 'exam'}}]
[{'name': 'Cody Strouth', 'scores': {'score': 99.80348240553108, 'type': 'quiz'}}]


# 2. Find students who scored below average in the exam and pass mark is 40%?

In [18]:
stage1 = {'$unwind':'$scores'}

stage2 = {'$match':{'scores.type':'exam'}}

stage3 = {'$group':{'_id':'$scores.type', 'Average':{'$avg':'$scores.score'}}}

stage4 = {'$project':{'_id':0,'name':1}}

    
exam_average = collection.aggregate([stage1,stage2,stage3])

exam_average = list(exam_average)[0]['Average']
exam_average

48.67367075950175

In [20]:
y = collection.aggregate([{'$unwind':'$scores'},
                        {'$match': {'$and': [{'scores.type':'exam'},{'scores.score':{'$gte':40,'$lt': exam_average}}]}},
                        {'$project' : {'name':1,'scores.type':1 }} ])
list(y)

[{'_id': 5, 'name': 'Wilburn Spiess', 'scores': {'type': 'exam'}},
 {'_id': 10, 'name': 'Denisha Cast', 'scores': {'type': 'exam'}},
 {'_id': 19, 'name': 'Gisela Levin', 'scores': {'type': 'exam'}},
 {'_id': 20, 'name': 'Tressa Schwing', 'scores': {'type': 'exam'}},
 {'_id': 21, 'name': 'Rosana Vales', 'scores': {'type': 'exam'}},
 {'_id': 23, 'name': 'Tamika Schildgen', 'scores': {'type': 'exam'}},
 {'_id': 29, 'name': 'Gwyneth Garling', 'scores': {'type': 'exam'}},
 {'_id': 42, 'name': 'Kayce Kenyon', 'scores': {'type': 'exam'}},
 {'_id': 45, 'name': 'Terica Brugger', 'scores': {'type': 'exam'}},
 {'_id': 57, 'name': 'Chad Rahe', 'scores': {'type': 'exam'}},
 {'_id': 68, 'name': 'Jenise Mcguffie', 'scores': {'type': 'exam'}},
 {'_id': 89, 'name': 'Cassi Heal', 'scores': {'type': 'exam'}},
 {'_id': 100, 'name': 'Demarcus Audette', 'scores': {'type': 'exam'}},
 {'_id': 109, 'name': 'Flora Duell', 'scores': {'type': 'exam'}},
 {'_id': 126, 'name': 'Quincy Danaher', 'scores': {'type': 'e

# 3. Find students who scored below pass mark and assigned them as fail, and above pass mark as pass in all the categories.Student who scored below 40%


In [21]:
# failed students in exam

stage1 = {'$unwind':'$scores'}

stage2 = {'$match': {'$and': [ {'scores.type':'exam'}, {'scores.score':{'$lt':40} } ] } }

failed_student_exm = collection.aggregate([stage1,stage2])

failed_students_exam = list(failed_student_exm)


for i in failed_students_exam:
    collection.update_one({'_id':i['_id']},
                    {'$push': {'Suject_Results' : {'exam':'Fail'}}})

In [23]:
# failed students in quiz

stage11 = {'$unwind':'$scores'}

stage22 = {'$match': {'$and': [ {'scores.type':'quiz'}, {'scores.score':{'$lt':40} } ] } }

failed_student_qiz = collection.aggregate([stage11,stage22])

failed_students_quiz = list(failed_student_qiz)
for i in failed_students_quiz:
    collection.update_one({'_id':i['_id']},
                    {'$push': {'Suject_Results' : {'quiz':'Fail'}}})

In [25]:
# failed students in homework

stage11 = {'$unwind':'$scores'}

stage22 = {'$match': {'$and': [ {'scores.type':'homework'}, {'scores.score':{'$lt':40} } ] } }

failed_student_hmwrk = collection.aggregate([stage11,stage22])

failed_students_home_work = list(failed_student_hmwrk)



for i in failed_students_home_work:
    collection.update_one({'_id':i['_id']},
                    {'$push': {'Suject_Results' : {'homework':'Fail'}}})

# Students got pass mark in each category

In [27]:
# students who passed in exam

stage1 = {'$unwind':'$scores'}

stage2 = {'$match': {'$and': [ {'scores.type':'exam'}, {'scores.score':{'$gt':40} } ] } }

passed_student_exm = collection.aggregate([stage1,stage2])

passed_students_exam = list(passed_student_exm)



for i in passed_students_exam:
    collection.update_one({'_id':i['_id']},
                    {'$push': {'Suject_Results' : {'exam':'pass'}}})

In [28]:
# students who passed in quiz

stage1 = {'$unwind':'$scores'}

stage2 = {'$match': {'$and': [ {'scores.type':'quiz'}, {'scores.score':{'$gt':40} } ] } }

passed_student_qiz = collection.aggregate([stage1,stage2])

passed_students_quiz = list(passed_student_qiz)




for i in passed_students_quiz:
    collection.update_one({'_id':i['_id']},
                    {'$push': {'Suject_Results' : {'quiz':'Pass'}}})

In [29]:
# students who passed in homework

stage1 = {'$unwind':'$scores'}

stage2 = {'$match': {'$and': [ {'scores.type':'homework'}, {'scores.score':{'$gt':40} } ] } }

passed_student_hmwrk = collection.aggregate([stage1,stage2])

passed_students_home_work = list(passed_student_hmwrk)

passed_students_home_work


for i in passed_students_home_work:
    collection.update_one({'_id':i['_id']},
                    {'$push': {'Suject_Results' : {'homework':'Pass'}}})

# 4. Find the total and average of the exam, quiz and homework and store them in a separate collection

In [30]:
stage1 = {'$unwind': '$scores'}

stage2 = {'$group': {'_id':'$name', 'total_marks': {'$sum': '$scores.score'} ,'avg_of_all' : {'$avg': '$scores.score'}}}

stage3 = {'$sort': {'_id':1} }

#stage3 = {'$project': {'_id':0}}

#stage4 = {'$out': 'Average_of_all_type'}

averafe_of_all_the_type = collection.aggregate([stage1,stage2,stage3])

list(averafe_of_all_the_type)

[{'_id': '',
  'total_marks': 158.1297878869123,
  'avg_of_all': 52.70992929563744},
 {'_id': 'Adrien Renda',
  'total_marks': 374.6246378987548,
  'avg_of_all': 62.43743964979247},
 {'_id': 'Aleida Elsass',
  'total_marks': 358.90349761860284,
  'avg_of_all': 59.81724960310047},
 {'_id': 'Alica Pasley',
  'total_marks': 334.3533678054747,
  'avg_of_all': 55.72556130091245},
 {'_id': 'Alix Sherrill',
  'total_marks': 200.76835837566375,
  'avg_of_all': 33.461393062610625},
 {'_id': 'Aurelia Menendez',
  'total_marks': 289.72548296495546,
  'avg_of_all': 48.28758049415924},
 {'_id': 'Bao Ziglar',
  'total_marks': 292.7546810241508,
  'avg_of_all': 48.79244683735846},
 {'_id': 'Barbera Lippman',
  'total_marks': 330.8387817789854,
  'avg_of_all': 55.13979696316423},
 {'_id': 'Beckie Millington',
  'total_marks': 129.29870077731468,
  'avg_of_all': 43.09956692577156},
 {'_id': 'Brain Lachapelle',
  'total_marks': 135.07862363042534,
  'avg_of_all': 45.02620787680845},
 {'_id': 'Brittny Wa

In [31]:
db.list_collection_names()

['Student_Database']

# 5) Create a new collection which consists of students who scored below average and above 40% in all the categories.

In [32]:
stage1 = {'$unwind':'$scores'}

#stage2 = {'$match':{'scores.type':'exam'}}

stage2 = {'$group':{'_id':'$scores.type', 'Average':{'$avg':'$scores.score'}}}

stage4 = {'$project':{'_id':0,'name':1}}
    
exam = collection.aggregate([stage1,stage2])

exam_average = list(exam)
exam_average

[{'_id': 'quiz', 'Average': 48.99672319430254},
 {'_id': 'homework', 'Average': 67.81869620661149},
 {'_id': 'exam', 'Average': 48.67367075950175}]

In [33]:
quiz_avg = exam_average[0]['Average']
exam_avg = exam_average[1]['Average']
homework_avg = exam_average[2]['Average']

print(quiz_avg,exam_avg,homework_avg)

48.99672319430254 67.81869620661149 48.67367075950175


In [34]:
# saved the data in the new collection
for i in exam_average:
    y =   collection.aggregate([{'$unwind':'$scores'},
                        {'$match': {'$and': [{'scores.type':i['_id']},{'scores.score':{'$gt':40 ,'$lt': i['Average']}}]}},
                         {'$project':{'_id':0}}#,
                        #{'$out':'Avg_of_all_exam_type'}
                           ])
    pprint(list(y))
    
    
db.list_collection_names()

[{'Suject_Results': [{'exam': 'pass'}, {'quiz': 'Pass'}, {'homework': 'Pass'}],
  'name': 'Salena Olmos',
  'scores': {'score': 42.48780666956811, 'type': 'quiz'}},
 {'Suject_Results': [{'homework': 'Fail'},
                     {'homework': 'Fail'},
                     {'exam': 'pass'},
                     {'quiz': 'Pass'}],
  'name': 'Terica Brugger',
  'scores': {'score': 41.73654145887228, 'type': 'quiz'}},
 {'Suject_Results': [{'exam': 'Fail'},
                     {'exam': 'pass'},
                     {'quiz': 'Pass'},
                     {'homework': 'Pass'}],
  'name': 'Danika Loeffler',
  'scores': {'score': 41.75962115078149, 'type': 'quiz'}},
 {'Suject_Results': [{'exam': 'Fail'},
                     {'exam': 'pass'},
                     {'quiz': 'Pass'},
                     {'homework': 'Pass'}],
  'name': 'Joaquina Arbuckle',
  'scores': {'score': 40.48858382583742, 'type': 'quiz'}},
 {'Suject_Results': [{'exam': 'Fail'},
                     {'homework': 'Fail'},
 

['Student_Database']

# 6. Create a new collection which consists of students who scored below the fail mark in all the categories

In [35]:
stage1 = {'$unwind':'$scores'}

stage2 = {'$match': {'scores.score':{'$lt':40}}}

for i in collection.aggregate([stage1,stage2]):
    print(i)
    
faill_in_all = collection.aggregate([stage1,stage2])

students_failed_in_all3 = list(faill_in_all)

students_failed_in_all3

{'_id': 0, 'name': 'aimee Zank', 'scores': {'score': 1.463179736705023, 'type': 'exam'}, 'Suject_Results': [{'exam': 'Fail'}, {'quiz': 'Fail'}, {'homework': 'Fail'}, {'homework': 'Fail'}, {'exam': 'pass'}]}
{'_id': 0, 'name': 'aimee Zank', 'scores': {'score': 11.78273309957772, 'type': 'quiz'}, 'Suject_Results': [{'exam': 'Fail'}, {'quiz': 'Fail'}, {'homework': 'Fail'}, {'homework': 'Fail'}, {'exam': 'pass'}]}
{'_id': 0, 'name': 'aimee Zank', 'scores': {'score': 35.8740349954354, 'type': 'homework'}, 'Suject_Results': [{'exam': 'Fail'}, {'quiz': 'Fail'}, {'homework': 'Fail'}, {'homework': 'Fail'}, {'exam': 'pass'}]}
{'_id': 2, 'name': 'Corliss Zuk', 'scores': {'score': 6.301851677835235, 'type': 'quiz'}, 'Suject_Results': [{'quiz': 'Fail'}, {'exam': 'pass'}, {'homework': 'Pass'}]}
{'_id': 3, 'name': 'Bao Ziglar', 'scores': {'score': 24.80221293650313, 'type': 'quiz'}, 'Suject_Results': [{'quiz': 'Fail'}, {'exam': 'pass'}, {'homework': 'Pass'}]}
{'_id': 4, 'name': 'Zachary Langlais', 's

[{'_id': 0,
  'name': 'aimee Zank',
  'scores': {'score': 1.463179736705023, 'type': 'exam'},
  'Suject_Results': [{'exam': 'Fail'},
   {'quiz': 'Fail'},
   {'homework': 'Fail'},
   {'homework': 'Fail'},
   {'exam': 'pass'}]},
 {'_id': 0,
  'name': 'aimee Zank',
  'scores': {'score': 11.78273309957772, 'type': 'quiz'},
  'Suject_Results': [{'exam': 'Fail'},
   {'quiz': 'Fail'},
   {'homework': 'Fail'},
   {'homework': 'Fail'},
   {'exam': 'pass'}]},
 {'_id': 0,
  'name': 'aimee Zank',
  'scores': {'score': 35.8740349954354, 'type': 'homework'},
  'Suject_Results': [{'exam': 'Fail'},
   {'quiz': 'Fail'},
   {'homework': 'Fail'},
   {'homework': 'Fail'},
   {'exam': 'pass'}]},
 {'_id': 2,
  'name': 'Corliss Zuk',
  'scores': {'score': 6.301851677835235, 'type': 'quiz'},
  'Suject_Results': [{'quiz': 'Fail'},
   {'exam': 'pass'},
   {'homework': 'Pass'}]},
 {'_id': 3,
  'name': 'Bao Ziglar',
  'scores': {'score': 24.80221293650313, 'type': 'quiz'},
  'Suject_Results': [{'quiz': 'Fail'},
 

# 7. Create a new collection which consists of students who scored above pass mark in all the categories

In [36]:
# Ids of the students who passed in each categories
all_pass = collection.aggregate([
                    {'$unwind': '$scores'},
                    {'$match' :{'scores.score':{'$gte': 40}}},
                    {'$group' :{'_id':{'_id':'$_id','name':'$name'},'count':{'$sum':1}}},
])

students_passed_in_all = list(all_pass)

In [37]:
# Creating a collection
std_allpass = db['std_allpass']

In [40]:
# filtering the Ids of the students who passed in all categories and uploading their information into a collection named 'std_allpass'
for i in students_passed_in_all:
    if i['count'] == 3:
        firstpair = i['_id']
        id1 = firstpair['_id']
        name = firstpair['name']
        std_allpass.insert_one({'_id':id1,'name':name})

In [41]:
students_passed_in_all

[{'_id': {'_id': 184, 'name': 'Harriett Velarde'}, 'count': 3},
 {'_id': {'_id': 126, 'name': 'Quincy Danaher'}, 'count': 3},
 {'_id': {'_id': 166, 'name': 'Tawana Oberg'}, 'count': 3},
 {'_id': {'_id': 173, 'name': 'Vinnie Auerbach'}, 'count': 2},
 {'_id': {'_id': 24, 'name': 'Jesusa Rickenbacker'}, 'count': 2},
 {'_id': {'_id': 195, 'name': 'Linnie Weigel'}, 'count': 2},
 {'_id': {'_id': 1, 'name': 'Aurelia Menendez'}, 'count': 3},
 {'_id': {'_id': 151, 'name': 'Meagan Oakes'}, 'count': 2},
 {'_id': {'_id': 96, 'name': 'Milan Mcgavock'}, 'count': 1},
 {'_id': {'_id': 150, 'name': 'Eugene Magdaleno'}, 'count': 1},
 {'_id': {'_id': 199, 'name': 'Rae Kohout'}, 'count': 2},
 {'_id': {'_id': 89, 'name': 'Cassi Heal'}, 'count': 2},
 {'_id': {'_id': 64, 'name': 'Whitley Fears'}, 'count': 3},
 {'_id': {'_id': 135, 'name': 'Rosana Vales'}, 'count': 1},
 {'_id': {'_id': 138, 'name': 'Jesusa Rickenbacker'}, 'count': 1},
 {'_id': {'_id': 190, 'name': 'Adrien Renda'}, 'count': 3},
 {'_id': {'_id'